In [56]:
import math
import numpy as np
from shapely.geometry import MultiPoint

In [57]:
breakpoints = [([-37.647863, 363.520713], 1), ([-86.993307, 566.306022], 1), ([-116.276208, 686.644227], 1), ([-43.256272, 704.133424], 0), ([-97.187671, 928.036095], 0), ([-30.823008, 943.939461], 1), ([-22.791816, 910.584813], 0), ([41.779946, 926.056381], 0), ([129.221734, 546.278255], 1)]

Cxy = list(range(58, 109))
Ch = [9, 12]

In [58]:
class BaseSection:
    def __init__(self, angle, leng, widt, road):
        self.angle = angle
        self.an_normalized = self.angle_normalized()
        self.length, self.width, self.road = leng, widt, road
        self.length_dep = self.length_dependency()
        self.length_r, self.width_r = self.possible_combinations()
        self.flat_length, self.flat_width = self.flat_dictionary()

    def angle_normalized(self):
        if self.angle > math.pi / 4:
            if self.angle < math.pi / 2:
                ang = (math.pi / 2) - self.angle
            elif 3 * math.pi / 4 > self.angle > math.pi / 2:
                ang = self.angle - (math.pi / 2)
            else:
                ang = math.pi - self.angle
        else:
            ang = self.angle
        return ang

In [271]:
class PolyClusters:
    def __init__(self, split_points):
        self.split_points = split_points
        self.bound_rec = np.asarray(MultiPoint([i[0] for i in self.split_points]).minimum_rotated_rectangle.exterior.coords)
        self.projection = self.points_projected()

    #считаем точки спроецированные на баундинг ректангл
    def points_projected(self):
        points = []
        for i, v in enumerate(self.bound_rec[0:4]):
            if i != len(self.bound_rec)-1:
                temp =[]
                length = np.sum((v-self.bound_rec[i+1])**2)
                for sp in self.split_points:
                    if sp[1] == 1:
                        t = max(0, min(1, np.sum((np.asarray(sp[0]) - v) * (self.bound_rec[i+1] - v)) / length))
                        projection = np.asarray(v + t * (self.bound_rec[i+1] - v))
                        temp.append(projection)
                    else:
                        pass
            points.append(np.asarray(temp))
        return np.asarray(points)

    def distances(self):
        arr = np.delete(self.projection, 4, axis=1)
        shift_arr = np.delete(np.roll(self.projection, -1, axis=1), 4, axis=1)
        dist = np.linalg.norm(arr-shift_arr, axis = 2 )
        dist_point = np.stack((arr, shift_arr), axis = 2)
        values = []
        for i in dist:
            d =[]
            sum = 0
            for v in range(len(i)):
                if i[v] < 58 and v!=len(i)-1:
                    sum+=i[v]
                elif i[v] < 58 and v==len(i)-1:
                    sum+=i[v]
                    d[-1]+=sum
                else:
                    d.append(sum + i[v])
                    sum = 0

            values.append(d)
        return values, dist













In [272]:
p_c = PolyClusters(breakpoints)
v, b = p_c.distances()

#print(p_c.projection)
v, b

([[[208.6867483082789, 0],
   [123.84027628145742, 1],
   [231.56154456452532, 2],
   [423.4319528758201, 3]],
  [[145.12783227425433, 2], [66.73930987798941, 3]],
  [[208.68674830827885, 0],
   [123.84027628145742, 1],
   [231.5615445645252, 2],
   [423.43195287581995, 3]],
  [[145.12783227425442, 2], [66.73930987798934, 3]]],
 array([[208.68674831, 123.84027628, 231.56154456, 423.43195288],
        [  2.58756069,   1.53552731, 141.00474428,  66.73930988],
        [208.68674831, 123.84027628, 231.56154456, 423.43195288],
        [  2.58756069,   1.53552731, 141.00474428,  66.73930988]]))

In [273]:
i = [7,5,60,72,1,5,74,2,4]

d =[]
sum = 0
for v in range(len(i)):
    if i[v] < 58 and v!=len(i)-1:
        sum+=i[v]
    elif i[v] < 58 and v==len(i)-1:
        sum+=i[v]
        d[-1]+=sum
    else:
        d.append(sum + i[v])
        sum = 0
d

[72, 72, 86]